In [1]:
import wandb, argparse
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback

path = untar_data(URLs.PETS)/'images'

In [2]:
def get_dls(bs, image_size, batch_tfms=None, pin_memory=False, num_workers=4):
    dataset_path = untar_data(URLs.PETS)
    files = get_image_files(dataset_path/"images")
    dls = ImageDataLoaders.from_name_re(
            dataset_path, files, r'(^[a-zA-Z]+_*[a-zA-Z]+)', valid_pct=0.2,
            seed=1234, bs=bs, item_tfms=Resize(image_size), batch_tfms=batch_tfms, 
            pin_memory=pin_memory, num_workers=num_workers)
    return dls

In [3]:
dls = get_dls(128, 224)

In [4]:
cbs = [SaveModelCallback(), 
       #the name you git in the WandbCallback overrides the one from the
       #SaveModelCallback, so don't pass anything
       WandbCallback(log_model=True, model_name="my_model")]  #it will put the .pth
learn = vision_learner(
    dls, 
    "regnetx_032", 
    metrics=error_rate, 
    cbs=cbs,
    path=".", #this saves the model in the local models/ folder
).to_fp16()

In [5]:
doc(WandbCallback)

,Type,Default,Details
log,str,None,"What to log (can be gradients, parameters, all or None)"
log_preds,bool,True,Whether to log model predictions on a wandb.Table
log_preds_every_epoch,bool,False,Whether to log predictions every epoch or at the end
log_model,bool,False,Whether to save the model checkpoint to a wandb.Artifact
model_name,str,None,"The name of the model_name to save, overrides SaveModelCallback"
log_dataset,bool,False,Whether to log the dataset to a wandb.Artifact
dataset_name,str,None,A name to log the dataset with
valid_dl,TfmdDL,None,"If log_preds=True, then the samples will be drawn from valid_dl"
n_preds,int,36,How many samples to log predictions
seed,int,12345,The seed of the samples drawn


In [6]:
with wandb.init(project='inference_fastai', job_type="train"):
    learn.fit(1)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


epoch,train_loss,valid_loss,error_rate,time
0,0.861552,0.244109,0.071042,00:17


Better model found at epoch 0 with valid_loss value: 0.24410924315452576.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
error_rate,▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,█▇▆▅▄▃▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sqr_mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sqr_mom_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


we can load the model back if we have a learner created

In [7]:
learn.load("my_model")

We can download the model from the artifact:

In [8]:
api = wandb.Api()
artifact = api.artifact('capecape/inference_fastai/my_model:latest')
artifact_dir = artifact.download()

wandb: Downloading large artifact my_model:latest, 58.96MB. 1 files... Done. 0:0:0


In [9]:
Path(artifact_dir).ls()

(#1) [Path('artifacts/my_model:v2/my_model.pth')]

it works 😎 (this guy wants a full path)

In [10]:
learn.load(Path(artifact_dir).absolute()/"my_model")

## Exporting the Learner

In [15]:
learn.export("learner.pkl")

In [16]:
at = wandb.Artifact("Model_learner", 
                    type="model", 
                    description="The fastai learner object")

In [17]:
at.add_file("learner.pkl")

<ManifestEntry digest: J/UhNxdIrht2oc6COvNjxg==>

In [18]:
with wandb.init(project='inference_fastai', job_type="save_learn"):
    wandb.log_artifact(at)

In [19]:
api = wandb.Api()
artifact = api.artifact('capecape/inference_fastai/Model_learner:latest')
artifact_dir = artifact.download()

wandb: Downloading large artifact Model_learner:latest, 59.22MB. 1 files... Done. 0:0:0


In [21]:
Path(artifact_dir).ls()

(#1) [Path('artifacts/Model_learner:v0/learner.pkl')]

wandb: ERROR Error while calling W&B API: json: cannot unmarshal string into Go value of type map[string]interface {} (<Response [500]>)


In [23]:
learn = load_learner(Path(artifact_dir)/"learner.pkl")

wandb: ERROR Error while calling W&B API: json: cannot unmarshal string into Go value of type map[string]interface {} (<Response [500]>)
